In [ ]:
'''Runs and saves sub models'''
## Import necessary functions
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import csv
import itertools
import math
import pickle
# from tkinter import filedialog #Only use if using file selecter

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
max_depth = 9 #default None
max_leaf = 20 #default 31
min_samples = 35 # default 20
test_size = 0.25
rand_state = 42

def name(df):
    '''df: dataframe 
    Make filename for model'''
    fl = ('_').join(sorted([i for i in df]))
    return fl
    
    
## Import Data

### Select file from computer to run
file_path = '../inc_csvs/adult_reconstruction.csv'
df =  pd.read_csv(file_path)
df= df[['Hours per Week', 'Age', 'Workclass', 'Highest Degree',
       'Marital Status', 'Race', 'Gender', 'Native Country', 'Occupation', 'Income']]
## Drop the income column

numFeatures = list(df.select_dtypes(include = 'number').drop(labels=['Income'], axis=1))
y = df[['Income']].copy().values.squeeze() # (49531,)
catFeatures = list(df.select_dtypes(include = object))

x = df[catFeatures + numFeatures].copy() # (49531, 13)
x[catFeatures] = x[catFeatures].astype("category")


## Fit training set to model

## Process categorical features for OneHotEncoder
one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
)

hist_one_hot = make_pipeline(
    one_hot_encoder, HistGradientBoostingClassifier(random_state=rand_state)
)
hist_one_hot = Pipeline([
("preprocess",one_hot_encoder), ("classifier", HistGradientBoostingClassifier(random_state=rand_state)),
]
)

### Check shape of feature data set
n_categorical_features = x.select_dtypes(include="category").shape[1]
n_numerical_features = x.select_dtypes(include="number").shape[1]


### Create test and train set from existing dataframe
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=rand_state)
# fullmodel = hist_one_hot.set_params(classifier__min_samples_leaf=min_samples,classifier__max_depth = max_depth, classifier__max_leaf_nodes = max_leaf).fit(x_train, y_train)


fullcat = catFeatures.copy()
fullnum = numFeatures.copy()
'''Use full model to test sub sets'''
num_feat = 3

df_short = list(itertools.combinations(list(df.drop(labels=['Income'], axis=1)), num_feat)) ##(df_short, num_feat) indicates combination of n features in each iteration

test_accs = []
train_accs = []
acc_dff = []
y_preds = []
y_probs = []
attributes = ['Workclass', 'Marital Status', 'Race', 'Gender', 'Occupation','Highest Degree','Native Country','Age','Hours per Week']
for combo in df_short:
    catFeatures = []
    numFeatures = []
    
    
    for feature in combo:
        if feature in ['Workclass', 'Marital Status', 'Race', 'Gender', 'Occupation','Highest Degree','Native Country']:#, 'native-country']:
            catFeatures.append(feature)
        elif feature in ['Hours per Week', 'Age']:
            numFeatures.append(feature)

    testdf = df.copy()

    for feature in [i for i in attributes if i not in combo]:
        if feature in fullnum:
            testdf[feature] = np.nan
        elif feature in fullcat:
            testdf[feature] = ''
            
            
    # testx = testdf[fullcat + fullnum].copy() # (49531, 13)
    # testx[fullcat] = testx[fullcat].astype("category")
    # _, testX_test, _, y_test = train_test_split(testx, y, test_size=test_size, random_state=rand_state)
    # # y_pred_full = cross_val_predict(fullmodel,testX_test,y_test,cv=5)
    # y_pred_full = fullmodel.predict(testX_test)
    # accs_full.append(fullmodel.score(testX_test,y_test))
    
    x = df[catFeatures + numFeatures].copy() # (49531, 13)
    x[catFeatures] = x[catFeatures].astype("category")
    y = df[['Income']].copy().values.squeeze() # (49531,)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=rand_state)
    submodel = hist_one_hot.set_params(classifier__min_samples_leaf=min_samples,classifier__max_depth = max_depth, classifier__max_leaf_nodes = max_leaf).fit(x_train, y_train)
    y_pred_sub = submodel.predict(x_test)
    accs_sub.append(submodel.score(x_test,y_test))
    ## make model that learns on restructed info
    # model,X_train, X_test, y_train, y_test = classmodel(x,y)
    # y_pred = cross_val_predict(model,X_test,y_test,cv=5)
    testacc,trainacc = average_roc(x,y,submodel)
    test_accs.append(testacc)
    train_accs.append(trainacc)
    '''Saving model'''
    fl = name(x) ##name specific to each version of each model
    # print(X_test.head())
    # save
    with open('./models/'+fl+'.pkl','wb') as f:
        pickle.dump(submodel,f)
# with open('./models/fullmodel.pkl', 'wb') as f:
#     pickle.dump(fullmodel,f)
# print(np.mean(accs_full),np.mean(accs_sub))
print(np.mean(test_accs),np.mean(train_accs))

/tmp/ipykernel_1369261/1986917733.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(10, 10))


### Test ability of each model

In [17]:
import os

models = os.listdir('./models')
for model in models:
    if 'fullmodel' in model:
        continue
    with open('./models/'+model, 'rb') as f:
        model = pickle.load(f)
        
def average_roc(x,y,model):
    cv = RepeatedStratifiedKFold(n_repeats=10, random_state=rand_state)
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    trainacc = []
    testacc = []
    fig, ax = plt.subplots(figsize=(10, 10))
    predarray = []
    for fold, (train, test) in enumerate(cv.split(x, y)):
        model.fit(x.iloc[train], y[train])
        trpred = model.predict(x.iloc[train])
        tepred = model.predict(x.iloc[test])
        trainacc.append((trpred==y[train]).sum()/len(trpred))
        testacc.append((tepred==y[test]).sum()/len(tepred))
        parr = np.full((45849),np.nan)
        # parr[train] = trpred
        parr[test] = tepred
        predarray.append(parr)
        # y_tepred = cross_val_predict(model, x.iloc[test], y[test], cv=3)
        viz = RocCurveDisplay.from_estimator(
            model,
            x.iloc[test],
            y[test],
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Mean ROC curve with variability (On Answers)",
    )
    ax.axis("square")
    ax.legend(loc="lower right")
    plt.gcf()
    handles, labels = plt.gca().get_legend_handles_labels()
    handles = handles[-3:]
    labels = labels[-3:]
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    # plt.show()
    if abs(np.mean(trainacc) - np.mean(testacc)) > 0.1:
        print(np.mean(trainacc),np.std(trainacc))
        print(np.mean(testacc),np.std(testacc)) 
    return np.mean(trainacc), np.mean(testacc)
    print(np.mean(trainacc),np.std(trainacc))
    print(np.mean(testacc),np.std(testacc))

In [11]:
df

,Hours per Week,Age,Workclass,Highest Degree,Marital Status,Race,Gender,Native Country,Occupation,Income Actual,Income Adjust,Income
0,20,40,Private,Undergraduate Degree,Married,White,Female,United States,Tech Support,49100,83475.473800,1
1,40,21,Private,High School Graduate,Divorced,White,Male,United States,Craft/Repair,11500,19551.282050,0
2,10,17,Private,Less than High School,Never Married,White,Male,United States,Other Services,2600,4420.289855,0
3,50,51,Private,High School Graduate,Married,Asian/Pacific Islander,Male,Cambodia,Sales,38997,66299.247490,1
4,38,26,Private,Undergraduate Degree,Never Married,White,Male,United States,Executive/Managerial,38524,65495.094760,1
...,...,...,...,...,...,...,...,...,...,...,...,...
45844,65,35,Private,Undergraduate Degree,Married,White,Male,Yugoslavia,Farming/Fishing,85080,144645.484900,1
45845,77,37,Self Employed (Unincorporated),Undergraduate Degree,Married,Asian/Pacific Islander,Male,Vietnam,Sales,34137,58036.705690,1
45846,55,24,Private,Undergraduate Degree,Never Married,White,Male,United States,Sales,13016,22128.651060,0
45847,40,24,Private,High School Graduate,Never Married,White,Female,United States,Administrative/Clerical,15000,25501.672240,0
